In [1]:
import networkx as nx
import matplotlib.pyplot as plt

# 1. Load up the directed multigraph

In [2]:
def load_directed_multigraph_from_txt_file(txt):
    """ Creates and returns a directed multigraph from the provided text file """
    
    G = nx.read_edgelist(txt, create_using=nx.MultiDiGraph(), nodetype = str, data=(("weight", int),))
    return G
    

    
G = load_directed_multigraph_from_txt_file("./enron.txt")
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 151
Number of edges: 266
Average in degree:   1.7616
Average out degree:   1.7616


# 2. How many unique sources did send an email? And how many emails were sent?

In [3]:
def get_number_of_unique_sources_that_sent_an_email(G):
    """ Returns the number of nodes in G, which is equal to the number of unique sources that sent an email"""
    
    return len(G)
    
    
def get_number_of_emails_sent_by_unique_sources(G):
    """ Returns the number of edges in G, which is equal to the number of emails sent by unique sources"""
    
    return G.number_of_edges()


num_of_nodes = get_number_of_unique_sources_that_sent_an_email(G)
num_of_edges = get_number_of_emails_sent_by_unique_sources(G)
print(F"Number of unique sources that did send an email: {num_of_nodes}")
print(F"Number of emails that were sent: {num_of_edges}")


Number of unique sources that did send an email: 151
Number of emails that were sent: 266


# 3. Assume that information to and from the research institute can only be exchanged through emails, When an email is sent to someone inside the institute, a communication channel is created, allowing the sender to provide information to the receiver, but not vice versa. Based on the emails sent on the data, is it possible for information to go from every sender to every other receiver?

In [4]:
""" If the network is strongly connected, i.e. there is a path between every pair of nodes, 
    then it is possible for information to go from every sender to every other reciever. 
    
    In this case, the graph is NOT strongly connected, therefore it is NOT possible
    for information to go from every sender to every other receiver
"""


print(F"Is this graph strongly connected? {nx.is_strongly_connected(G)}")

Is this graph strongly connected? False


# 4. Now assume that a communication channel established by an email allows information to be exchanged both ways. Based on the emails sent on the data, is it possible for information to go from every sender to every other receiver?

In [5]:
""" If the network is weakly connected, i.e. if replacing all of the network's directed edges with undirected edges
    results in a connected undirected graph, then it is possible for information to go from every sender
    to every other reciever.
    
    In this case, the graph IS weakly connected, therefore it is possible 
    for information to go from every sender to every reciever
"""

print(F"Is this graph weakly connected? {nx.is_weakly_connected(G)}")

Is this graph weakly connected? True


# 5. How many nodes are in the largest (in terms of nodes) weakly connected component?

In [6]:
largest_weakly_connected_component =  max(nx.weakly_connected_components(G), key=len) # Find the largest dictionary
num_of_nodes = len(largest_weakly_connected_component) # The length of that dictionary will be the number of nodes
print(F"There are {num_of_nodes} nodes in the largest weakly connected component")

There are 151 nodes in the largest weakly connected component


# 6. How many nodes are in the largest (in terms of nodes) strongly connected component?

In [7]:
largest_strongly_connected_component = max(nx.strongly_connected_components(G), key=len) # Find the largest dictionary
num_of_nodes = len(largest_strongly_connected_component) # The length of that dictionary will be the number of nodes
print(F"There are {num_of_nodes} nodes in the largest strongly connected component")

There are 57 nodes in the largest strongly connected component


# 7. Using the function subgraph, find the subgraph of nodes in a largest strongly connected component. Call this graph G_sc. How many nodes are in this graph? 

In [8]:
G_sc = G.subgraph(largest_strongly_connected_component)
num_of_nodes = len(G_sc)

print(nx.info(G_sc))
print(F"\nThere are {num_of_nodes} nodes in the G_sc")

Name: 
Type: MultiDiGraph
Number of nodes: 57
Number of edges: 112
Average in degree:   1.9649
Average out degree:   1.9649

There are 57 nodes in the G_sc


# 8. What is the average distance between nodes in G_sc?

In [9]:
print(F"The average distance between the nodes in G_sc is {nx.average_shortest_path_length(G_sc)}")

The average distance between the nodes in G_sc is 4.791666666666667


# 9. What is the largest possible distance between two employees in G_sc?

In [10]:
print(F"The largest possible distance between two employees in G_sc is {nx.diameter(G_sc)}")

The largest possible distance between two employees in G_sc is 12


# 10. What is the set of nodes in G_sc with eccentricity equal to the diameter?

In [11]:
# This list comprehension returns the nodes with eccentricity equal to the diameter
set_of_nodes_with_eccentricity_equal_to_diameter = [
    node for node in nx.eccentricity(G_sc) if nx.eccentricity(G_sc)[node] == nx.diameter(G_sc)
]
    
print(F"The set of nodes in G_sc with eccentricity equal to the diameter is:\n {set_of_nodes_with_eccentricity_equal_to_diameter}")

The set of nodes in G_sc with eccentricity equal to the diameter is:
 ['343253', '343120']


# 11. What is the set of node(s) in G_sc with eccentricity equal to the radius? How many nodes are connected to this node?

In [12]:
#This list will returns the nodes with an eccentricity equal to the radius
set_of_nodes_with_eccentricity_equal_to_radius = [
    node for node in nx.eccentricity(G_sc) if nx.eccentricity(G_sc)[node] == nx.radius(G_sc)]
print(F"The set of nodes in G_sc with eccentricity equal to the radius is:\n {set_of_nodes_with_eccentricity_equal_to_radius}")

#calls out the how many nodes are connected via to the specified node
d = nx.radius(G_sc)
peripheries = nx.periphery(G_sc)
max_count = -1
result_node = None

for node in peripheries:
    count = 0
    sp = nx.shortest_path_length(G, node)
    for key, value in sp.items():
            if value == d:
                count += 1
                
            if count > max_count:
                result_node = node
                max_count = count
print(F"There are {max_count} nodes connected to this set.")

The set of nodes in G_sc with eccentricity equal to the radius is:
 ['343119']
There are 3 nodes connected to this set.


# 12. Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc? How many nodes are connected to this node?

In [13]:
#calls out the how many nodes are connected via to the specified node that is equal to the diameter
d = nx.diameter(G_sc)
peripheries = nx.periphery(G_sc)
max_count = -1
result_node = None

for node in peripheries:
    count = 0
    sp = nx.shortest_path_length(G, node)
    for key, value in sp.items():
            if value == d:
                count += 1
                
            if count > max_count:
                result_node = node
                max_count = count
print(F"{result_node} is connected to the most nodes by the shortest path of length.")
print(F"There are {max_count} nodes connected to this set.")

343253 is connected to the most nodes by the shortest path of length.
There are 1 nodes connected to this set.


# 13. Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)?

In [14]:
center = nx.center(G_sc)[0] # center = 343119
print(center)
node = result_node[0] # node = 3; this node does not exist; there are no single-digit nodes in enron.txt
print(node)
len(nx.minimum_node_cut(G_sc, center, node))

343119
3


NetworkXError: node 3 not in graph

# 14. Construct an undirected graph G_un using G_sc (you can ignore the attributes).

In [15]:
G = G_sc
undir_subgraph = G.to_undirected()
G_un = nx.Graph(undir_subgraph) 
print(nx.info(G_un))

Name: 
Type: Graph
Number of nodes: 57
Number of edges: 104
Average degree:   3.6491


# 15. What is the transitivity and average clustering coefficient of graph G_un?

In [16]:
print(nx.transitivity(G_un))
print(nx.average_clustering(G_un))

0.11214953271028037
0.12094017094017093
